In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
from pyspark.sql.functions import col, array, explode_outer, from_json, when, collect_set, array_join

### Initialize Spark Session

In [2]:
# Initialize Spark session
spark = SparkSession.builder.appName("BigDataAnalyticsPipeline").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 10:45:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Large Data Ingestion

In [3]:
# Define the base directory for the project
base_dir = "./data"

# Gather paths to all text files that are actually JSON
file_paths = []
for root, dirs, files in os.walk(base_dir):

    for file in files:
        # Add only text files (assuming they don’t have a .json extension)
        if not file.endswith('.json'):
            file_paths.append(os.path.join(root, file))

# Load all JSON text files into a single Spark DataFrame
# Using the option `multiLine=True` if each file contains multiple JSON objects across lines
df = spark.read.option("multiLine", True).json(path=file_paths)

# Display schema and sample records to verify
df.printSchema()
df.show(5, truncate=True)

24/11/12 10:45:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
24/11/12 10:46:35 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


root
 |-- abstracts-retrieval-response: struct (nullable = true)
 |    |-- affiliation: string (nullable = true)
 |    |-- authkeywords: struct (nullable = true)
 |    |    |-- author-keyword: string (nullable = true)
 |    |-- authors: struct (nullable = true)
 |    |    |-- author: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- @_fa: string (nullable = true)
 |    |    |    |    |-- @auid: string (nullable = true)
 |    |    |    |    |-- @seq: string (nullable = true)
 |    |    |    |    |-- affiliation: string (nullable = true)
 |    |    |    |    |-- author-url: string (nullable = true)
 |    |    |    |    |-- ce:alias: string (nullable = true)
 |    |    |    |    |-- ce:alt-name: string (nullable = true)
 |    |    |    |    |-- ce:degrees: string (nullable = true)
 |    |    |    |    |-- ce:given-name: string (nullable = true)
 |    |    |    |    |-- ce:indexed-name: string (nullable = true)
 |    |    |    |    |

+----------------------------+
|abstracts-retrieval-response|
+----------------------------+
|        {[{"affiliation-c...|
|        {[{"affiliation-c...|
|        {[{"affiliation-c...|
|        {[{"affiliation-c...|
|        {[{"affiliation-c...|
+----------------------------+
only showing top 5 rows



### Data Transformation

In [4]:
# Define the affiliation schema
affiliation_schema = StructType([
    StructField("country", StringType(), True),
    StructField("city", StringType(), True),
])

# Define the author-group schema
author_group_element_schema = StructType([
    StructField("affiliation", affiliation_schema, True),
])

author_group_array_schema = ArrayType(author_group_element_schema)
# Extract title, abstract, publication date, and affiliation
df = df.withColumn(
    "author_group_string",
    col("abstracts-retrieval-response.item.bibrecord.head.author-group").cast("string")
)

df = df.withColumn(
    "author_group_parsed_array",
    from_json(col("author_group_string"), author_group_array_schema)
).withColumn(
    "author_group_parsed_single",
    from_json(col("author_group_string"), author_group_element_schema)
)

df = df.withColumn(
    "author_group_array",
    when(col("author_group_parsed_array").isNotNull(), col("author_group_parsed_array"))
    .when(col("author_group_parsed_single").isNotNull(), array(col("author_group_parsed_single")))
    .otherwise(array())
).drop("author_group_string", "author_group_parsed_array", "author_group_parsed_single")

# Explode 'author_group_array' and extract 'country' and 'city'
author_affiliation_df = df.select(
    col("abstracts-retrieval-response.coredata.dc:identifier").alias("document_id"),
    explode_outer("author_group_array").alias("author_group")
).select(
    "document_id",
    col("author_group.affiliation.country").alias("affiliation_country"),
    col("author_group.affiliation.city").alias("affiliation_city")
)

# Aggregate countries and cities per document_id
documents_countries_df = author_affiliation_df.groupBy("document_id").agg(
    array_join(collect_set("affiliation_country"), "; ").alias("affiliation_countries"),
    array_join(collect_set("affiliation_city"), "; ").alias("affiliation_cities")
)

selected_df = df.select(
    col("abstracts-retrieval-response.coredata.dc:identifier").alias("document_id"),
    col("abstracts-retrieval-response.coredata.dc:title").alias("title"),
    col("abstracts-retrieval-response.coredata.dc:description").alias("description"),
    col("abstracts-retrieval-response.coredata.prism:coverDate").alias("publication_date"),
    col("abstracts-retrieval-response.coredata.dc:publisher").alias("publisher"),
)

final_df = documents_countries_df.join(
    selected_df,
    on="document_id",
    how="left"
)

# Show the final DataFrame
final_df.show(5, truncate=False)


+---------------------+--------------------------------+-------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
output_path = "documents_output"

# Write the final DataFrame to a CSV file
final_df.coalesce(1).write.csv(output_path, header=True, mode="overwrite")
